### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

In [7]:
from scipy.optimize import minimize
from numpy import linspace
pstaw = 10 ** (8.07131 - (1730.60/(20 + 233.426)))
psatd = 10 ** (7.43155 - (1554.679/(20 + 240.337)))
x1knob = linspace(0, 1, 11)
for i in x1knob:
    x2knob = 1 - x1knob
    press = lambda x: (x1knob*exp(x[0]*((x[0]*x2knob)/(x[0]*x1knob + x[1]*x2knob)) ** 2) * psatw + 
                      (x2knob*exp(x[1]*((x[0]*x1knob)/(x[0]*x1knob + x[1]*x2knob)) ** 2)* psatd))
    cons ({'type': 'eq' : x1knob + x2knob - 1})
    res = minimize(press, (1, 1), method='SLSQP',bounds=None, constraints=cons)
    res

SyntaxError: invalid syntax (<ipython-input-7-48b05200f7b4>, line 10)

In [25]:
# A simple example of using PyTorch for gradient descent
import numpy
import torch as t
from scipy import exp as exp
from scipy import gradient
from torch.autograd import Variable

psatw = 10 ** (8.07131 - (1730.60/(20 + 233.426)))
psatd = 10 ** (7.43155 - (1554.679/(20 + 240.337)))
x1knob = numpy.array([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
x2knob = 1 - x1knob
p = numpy.array([28.1, 34.4, 36.7, 36.9, 36.8,36.7, 36.5, 35.4, 32.9, 27.7, 17.5])
x = Variable(t.tensor([1.0, 0.0]), requires_grad=True)

# Fix the step size
a = 0.001

# Start gradient descent
for i in range(1000):  # TODO: change the termination criterion
    for i in range(0, len(x1knob)):
        loss = (x1knob*exp(x[0]*((x[0]*x2knob)/(x[0]*x1knob + x[1]*x2knob)) ** 2) * psatw + (x2knob*exp(x[1]*((x[0]*x1knob)/(x[0]*x1knob + x[1]*x2knob)) ** 2)* psatd) - p[i]) ** 2
        loss.backward()
    x.gradient.scipy()
    # no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to x
    with t.no_grad():
        x -= a * x.gradient
        
        # need to clear the gradient at every step, or otherwise it will accumulate...
        x.gradient.zero_()
        
print(x.data.numpy())
print(loss.data.numpy())
# Define a loss


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$.

In [4]:

def black_func(x, y):
     return 1/(4 - 2.1 * x ** 2 + ((x ** 4) / 3) * x ** 2 + x * y + (-4 + 4 * y ** 2) * y ** 2)
    
from bayes_opt import BayesianOptimization

bounds = {'x': (-3,3), 'y': (-2,2)}

optim = BayesianOptimization(f=black_func, pbounds=bounds, random_state=1)
optim.maximize(init_points=2,n_iter=10)
print(optim.max)

for i, res in enumerate(optim.res):
    print("Iteration {}: \n\t{}".format(i, res))

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        |  0.4252   | -0.4979   |  0.8813   |
|  2        |  0.004363 | -2.999    | -0.7907   |
|  3        |  0.3117   | -0.5814   |  1.057    |
|  4        |  0.3422   | -0.3102   |  0.4876   |
|  5        |  0.04594  | -2.104    |  0.5152   |
|  6        |  0.3345   | -0.3142   |  0.4566   |
|  7        |  0.2149   |  0.1121   |  1.061    |
|  8        |  0.7885   | -0.84     |  0.5745   |
|  9        |  0.7084   | -1.071    |  0.3112   |
|  10       |  8.34     | -1.183    |  0.7247   |
|  11       |  8.948    | -1.295    |  0.8243   |
|  12       |  0.04111  | -2.109    | -0.3733   |
{'target': 8.947749691096538, 'params': {'x': -1.294970077603155, 'y': 0.8243099161829229}}
Iteration 0: 
	{'target': 0.4251709708452254, 'params': {'x': -0.4978679717845562, 'y': 0.8812979737686324}}
Iteration 1: 
	{'target': 0.004362821965838803, 'params': {'x': -2.9993137510959307, 'y': -0.790669